In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ortho_group
import open3d as o3d
import copy
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def transformation(seed=None):
    if seed == None:
        pass
    else: 
        np.random.seed(seed)

    rotation = ortho_group.rvs(dim=3)
    tranlate = np.random.rand(3)
    transform = np.eye(4)
    transform[:3, :3] = rotation
    transform[:3, 3] = tranlate

    print(transform)

    return transform

In [3]:
pcd1 = o3d.io.read_point_cloud("selected_pcds/pointcloud_0000.pcd")
pcd2 = o3d.io.read_point_cloud("selected_pcds/pointcloud_0004.pcd")

In [4]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd1, pcd2, 0.05, transformation(42))

[[ 0.22081075 -0.29306608  0.93024453  0.18182497]
 [ 0.6770521  -0.64047179 -0.36248634  0.18340451]
 [ 0.70202783  0.70986489  0.05699795  0.30424224]
 [ 0.          0.          0.          1.        ]]


In [5]:
print(reg_p2p.transformation)
print(reg_p2p.fitness)
print(reg_p2p.inlier_rmse)

[[ 0.19243586 -0.34810202  0.91749301  0.15078783]
 [ 0.67867441 -0.62809781 -0.38064968  0.17033794]
 [ 0.70878028  0.69592968  0.11537939  0.29089969]
 [ 0.          0.          0.          1.        ]]
0.015361117172157976
0.031172835853675297


In [6]:
def best_hyper(pcd1, pcd2):
    thresh = [0.05, 0.1, 0.2, 0.4]
    init = ["orthogonal", "ransac"]
    max_iters = [100, 200, 500]

    results = []

    best_error = 1e6
    curr_idx = 0
    best_idx = 0
    best_transformation = transformation()

    for i in thresh:
        for j in max_iters:
            for k in init:
                if k == "orthogonal":
                    init_trans = transformation()
                elif k == "ransac":
                    voxel_size = 0.05

                    src_ds = pcd1.voxel_down_sample(voxel_size)
                    tar_ds = pcd2.voxel_down_sample(voxel_size)

                    src_ds.estimate_normals(
                        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn = 20))
                    tar_ds.estimate_normals(
                        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn = 20))
                    
                    src_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
                        src_ds,
                        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*5, max_nn=50))
                    tar_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
                        tar_ds,
                        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*5, max_nn=50))
                    
                    init_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                        src_ds, tar_ds,
                        src_fpfh, tar_fpfh,
                        mutual_filter=True,
                        max_correspondence_distance=voxel_size * 1.5,
                        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
                        ransac_n=3,
                        checkers=[
                            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(voxel_size * 1.5)
                        ],
                        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
                    )

                    init_trans = init_result.transformation
                
                hyp_reg_p2p = o3d.pipelines.registration.registration_icp(pcd1, pcd2, 0.05, init_trans)
                evaluation_initial = o3d.pipelines.registration.evaluate_registration(pcd1, pcd2, i, init_trans)
                evaluation_icp = o3d.pipelines.registration.evaluate_registration(pcd1, pcd2, i, hyp_reg_p2p.transformation)

                if evaluation_icp.inlier_rmse< best_error:
                    best_error = evaluation_icp.inlier_rmse
                    best_idx = curr_idx
                    best_transformation = evaluation_icp.transformation

                results.append({
                    "Thresh": i,
                    "Max Iter": j,
                    "Initialization": k,
                    "Initial Fitness": evaluation_initial.fitness,
                    "Initial RMSE": evaluation_initial.inlier_rmse,
                    "ICP Fitness": evaluation_icp.fitness,
                    "ICP RMSE": evaluation_icp.inlier_rmse
                })

                curr_idx+=1
    
    return best_transformation, best_idx, results

        

In [7]:
best_transformation, best_idx, results = best_hyper(pcd1, pcd2)
results_df = pd.DataFrame(results)
results_df

[[ 0.44121921 -0.71729698  0.53926862  0.78517596]
 [ 0.66394725 -0.14335508 -0.73390965  0.19967378]
 [-0.60373807 -0.68186095 -0.41299635  0.51423444]
 [ 0.          0.          0.          1.        ]]
[[-0.60880873 -0.72508432 -0.32187678  0.03438852]
 [ 0.42016071  0.04945008 -0.90610136  0.9093204 ]
 [-0.67291672  0.68688239 -0.2745463   0.25877998]
 [ 0.          0.          0.          1.        ]]
[[-0.34110341 -0.53087909  0.77576791  0.27134903]
 [ 0.89598399  0.06603208  0.4391497   0.82873751]
 [ 0.28436096 -0.84487109 -0.4531354   0.35675333]
 [ 0.          0.          0.          1.        ]]
[[ 0.29803083 -0.94356094 -0.14446586  0.86310343]
 [-0.33399384 -0.24485506  0.91021652  0.62329813]
 [ 0.89421795  0.22302188  0.3881179   0.33089802]
 [ 0.          0.          0.          1.        ]]
[[-0.24636351 -0.48018499 -0.84185948  0.71324479]
 [ 0.93993348  0.09339208 -0.32833364  0.76078505]
 [ 0.2362839  -0.87218134  0.42833355  0.5612772 ]
 [ 0.          0.          

,Thresh,Max Iter,Initialization,Initial Fitness,Initial RMSE,ICP Fitness,ICP RMSE
0,0.05,100,orthogonal,0.008161,0.037088,0.011215,0.032619
1,0.05,100,ransac,0.984901,0.014183,0.986166,0.012467
2,0.05,200,orthogonal,0.009383,0.034817,0.011783,0.029313
3,0.05,200,ransac,0.985599,0.014247,0.986166,0.012467
4,0.05,500,orthogonal,0.037399,0.031518,0.046171,0.031595
5,0.05,500,ransac,0.979402,0.021844,0.986166,0.012466
6,0.10,100,orthogonal,0.052324,0.065367,0.062099,0.062406
7,0.10,100,ransac,0.997425,0.020588,0.997731,0.014403
8,0.10,200,orthogonal,0.050404,0.070683,0.062055,0.057197
9,0.10,200,ransac,0.997600,0.015749,0.997818,0.014430


In [8]:
results[best_idx]

{'Thresh': 0.05,
 'Max Iter': 500,
 'Initialization': 'ransac',
 'Initial Fitness': 0.9794021383373336,
 'Initial RMSE': 0.02184410302852095,
 'ICP Fitness': 0.9861662666375737,
 'ICP RMSE': 0.012466237826299884}

In [9]:
best_transformation

array([[ 9.99999814e-01, -6.09625237e-04, -1.99021302e-05,
        -9.19777564e-05],
       [ 6.09625052e-04,  9.99999814e-01, -9.31877409e-06,
        -8.71820512e-04],
       [ 1.99078075e-05,  9.30663952e-06,  1.00000000e+00,
        -5.65106777e-07],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [10]:
pcd1_transformed = copy.deepcopy(pcd1)
pcd1_transformed.transform(best_transformation)

pcd1_transformed.paint_uniform_color([1, 0, 0])
pcd2.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([pcd1_transformed, pcd2], 
                                  window_name="Transformed Point Cloud",
                                  width=800, height=600)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
